***GENERATED CODE FOR decisiontree PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv("hdfs://34.83.203.43:54310"+eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Occupation', 'transformation_label': 'String Indexer'}], 'feature': 'Occupation', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '31', 'mean': '', 'stddev': '', 'min': 'Agricultural and food scientists', 'max': 'Urban and regional planners', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Occupation'}, {'feature_label': 'Occupation', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Occupation')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'All_weekly', 'transformation_label': 'String Indexer'}], 'feature': 'All_weekly', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '31', 'mean': '903.63', 'stddev': '313.16', 'min': '1051', 'max': 'Na', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'All_weekly'}, {'feature_label': 'All_weekly', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('All_weekly')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'M_weekly', 'transformation_label': 'String Indexer'}], 'feature': 'M_weekly', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '31', 'mean': '950.69', 'stddev': '340.39', 'min': '1009', 'max': 'Na', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'M_weekly'}, {'feature_label': 'M_weekly', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('M_weekly')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'F_weekly', 'transformation_label': 'String Indexer'}], 'feature': 'F_weekly', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '31', 'mean': '730.6', 'stddev': '300.92', 'min': '1215', 'max': 'Na', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'F_weekly'}, {'feature_label': 'F_weekly', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('F_weekly')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split


def gradientboostingclassifier(df, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    Decision_tree_classifier = DecisionTreeClassifier()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    Decision_tree_classifier.fit(X_train, y_train)
    display(" Accuracy of Model : %s" %
            Decision_tree_classifier.score(X_test, y_test))

    data = {'model': Decision_tree_classifier,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
#%run decisiontreeHooks.ipynb
try:
	#sourcePreExecutionHook()

	predicthighestincometrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/demo/PredictHighestIncomeTrain.csv', 'filename': 'PredictHighestIncomeTrain.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line'}")
	#sourcePostExecutionHook(predicthighestincometrain)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run decisiontreeHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(predicthighestincometrain,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Occupation", "transformation_label": "String Indexer"}], "feature": "Occupation", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "31", "mean": "", "stddev": "", "min": "Agricultural and food scientists", "max": "Urban and regional planners", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Occupation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "All_workers", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "31", "mean": "174.87", "stddev": "252.26", "min": "3", "max": "948", "missing": "0"}, "updatedLabel": "All_workers"}, {"transformationsData": [{"feature_label": "All_weekly", "transformation_label": "String Indexer"}], "feature": "All_weekly", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "31", "mean": "903.63", "stddev": "313.16", "min": "1051", "max": "Na", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "All_weekly"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "M_workers", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "31", "mean": "93.65", "stddev": "130.92", "min": "3", "max": "570", "missing": "0"}, "updatedLabel": "M_workers"}, {"transformationsData": [{"feature_label": "M_weekly", "transformation_label": "String Indexer"}], "feature": "M_weekly", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "31", "mean": "950.69", "stddev": "340.39", "min": "1009", "max": "Na", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "M_weekly"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "F_workers", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "31", "mean": "81.16", "stddev": "154.49", "min": "0", "max": "773", "missing": "0"}, "updatedLabel": "F_workers"}, {"transformationsData": [{"feature_label": "F_weekly", "transformation_label": "String Indexer"}], "feature": "F_weekly", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "31", "mean": "730.6", "stddev": "300.92", "min": "1215", "max": "Na", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "F_weekly"}, {"feature": "Occupation_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "31", "mean": "15.0", "stddev": "9.09", "min": "0.0", "max": "30.0", "missing": "0"}, "updatedLabel": "Occupation_stringindexer_..."}, {"feature": "All_weekly_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "31", "mean": "6.13", "stddev": "6.6", "min": "0.0", "max": "19.0", "missing": "0"}, "updatedLabel": "All_weekly_stringindexer_..."}, {"feature": "M_weekly_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "31", "mean": "4.39", "stddev": "5.48", "min": "0.0", "max": "16.0", "missing": "0"}, "updatedLabel": "M_weekly_stringindexer_tr..."}, {"feature": "F_weekly_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "31", "mean": "1.77", "stddev": "3.1", "min": "0.0", "max": "10.0", "missing": "0"}, "updatedLabel": "F_weekly_stringindexer_tr..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run decisiontreeHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=DecisionTreeClassifier(autofe, ["Occupation_stringindexer", "All_workers", "M_workers", "M_weekly_stringindexer", "F_workers", "F_weekly_stringindexer"], "All_weekly_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
